In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [4]:
df=pd.read_csv('belgium_data.csv')
df

,Unnamed: 0,Exposure,DriverAge,Gender,VehiculeUsage,Sinistre
0,0,1.000000,50,0,1,1
1,1,1.000000,64,1,1,0
2,2,1.000000,60,0,1,0
3,3,1.000000,77,0,1,0
4,4,0.046575,28,1,1,1
...,...,...,...,...,...,...
163207,163207,1.000000,37,0,0,0
163208,163208,1.000000,44,0,1,0
163209,163209,1.000000,50,0,1,0
163210,163210,1.000000,43,0,1,0


In [5]:
class ManualLogisticRegression:
    """
    Implémentation manuelle de la régression logistique avec exposition
    """
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None
    
    def sigmoid(self, z):
        """
        Fonction d'activation sigmoid: 1/(1 + e^(-z))
        """
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X, y, exposure):
        """
        Entraîne le modèle avec descente de gradient en multipliant 
        la sigmoid par l'exposition
        
        Args:
            X: Features d'entraînement
            y: Cible d'entraînement
            exposure: Exposition au risque (valeur entre 0 et 1)
        """
        if exposure is None:
            raise ValueError("L'exposition est requise pour l'entraînement")
        
        # Initialiser les paramètres
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        for i in range(self.n_iterations):
            # Calculer la sortie du modèle (logits)
            z = np.dot(X, self.weights) + self.bias
            
            # P(y=1|x) × exposure
            sigmoid_values = self.sigmoid(z)
            y_pred = sigmoid_values * exposure
            
            # Pour tenir compte de l'exposition dans la dérivée
            # La dérivée de (sigmoid(z) * exposure) : exposure * sigmoid(z) * (1 - sigmoid(z))
            sigmoid_derivative = exposure * sigmoid_values * (1 - sigmoid_values)
            
            # Calculer les gradients pour w et b
            dw = (1/n_samples) * np.dot(X.T, (y_pred - y * exposure))
            db = (1/n_samples) * np.sum(y_pred - y * exposure)
            
            # Mettre à jour les paramètres
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
    
    def predict_proba(self, X, exposure=None):
        """
        Prédit les probabilités
        
        Args:
            X: Features
            exposure: Exposition (si None, retourne les probabilités brutes)
        """
        z = np.dot(X, self.weights) + self.bias
        proba = self.sigmoid(z)
        
        # Si l'exposition est fournie, multiplier les probabilités
        if exposure is not None:
            return proba * exposure
        return proba
    
    def predict(self, X, exposure=None, threshold=0.5):
        """
        Prédit les classes (0 ou 1)
        
        Args:
            X: Features
            exposure: Exposition (si None, utilise les probabilités brutes)
            threshold: Seuil de décision
        """
        probas = self.predict_proba(X, exposure)
        return (probas > threshold).astype(int)


In [6]:
def prepare_data(data, features, target):
    """
    Prépare les données pour l'entraînement
    
    Args:
        data: DataFrame contenant les données
        features: Liste des features à utiliser
        target: Nom de la colonne cible
        
    Returns:
        Données divisées pour l'entraînement et le test
    """
    # Vérifier si Exposure existe
    if "Exposure" not in data.columns:
        raise ValueError("La colonne 'Exposure' est requise")
    
    # Extraire les features et cibles
    X = data[features].values
    y = data[target].values
    
    # Mémoriser les indices originaux avant la division
    original_indices = data.index.tolist()
    indices = np.arange(len(original_indices))
    
    # Division train/test avec les indices
    train_indices, test_indices, y_train, y_test = train_test_split(
        indices, y, test_size=0.2, random_state=42
    )
    
    # Récupérer les features d'entraînement et de test
    X_train = X[train_indices]
    X_test = X[test_indices]
    
    # Standardisation des features (important pour la descente de gradient)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Récupérer les expositions
    exposure_train = data["Exposure"].iloc[train_indices].values
    exposure_test = data["Exposure"].iloc[test_indices].values
    
    return X_train, X_test, y_train, y_test, exposure_train, exposure_test, scaler



In [7]:
X_train, X_test, y_train, y_test, exposure_train, exposure_test, scaler = prepare_data(df, ['DriverAge', 'Gender', 'VehiculeUsage'], 
                                                                                       "Sinistre")

In [8]:
exposure_test

array([1., 1., 1., ..., 1., 1., 1.], shape=(32643,))

In [9]:
X_test

array([[-1.28079895, -0.60044408,  0.22491458],
       [ 0.60657555, -0.60044408,  0.22491458],
       [-1.61783011, -0.60044408,  0.22491458],
       ...,
       [ 2.08951266, -0.60044408,  0.22491458],
       [-0.134893  , -0.60044408,  0.22491458],
       [-1.14598648,  1.66543403,  0.22491458]], shape=(32643, 3))

In [10]:
def train_model(X_train, y_train, exposure_train, learning_rate=0.01, n_iterations=1000):
    """
    Entraîne un modèle de régression logistique manuel
    avec multiplication directe de la sigmoid par l'exposition
    
    Args:
        X_train: Features d'entraînement
        y_train: Cible d'entraînement
        exposure_train: Exposition (obligatoire)
        learning_rate: Taux d'apprentissage pour la descente de gradient
        n_iterations: Nombre d'itérations pour la descente de gradient
    
    Returns:
        Le modèle entraîné
    """
    # Créer et entraîner le modèle
    model = ManualLogisticRegression(learning_rate=learning_rate, n_iterations=n_iterations)
    model.fit(X_train, y_train, exposure_train)
    
    # Afficher les coefficients du modèle
    print("Coefficients du modèle:")
    print(f"  Biais: {model.bias:.6f}")
    for i, weight in enumerate(model.weights):
        print(f"  Feature {i+1}: {weight:.6f}")
    
    return model

In [18]:
model = train_model(X_train, y_train, exposure_train, learning_rate=0.01, n_iterations=10000)

Coefficients du modèle:
  Biais: -2.034463
  Feature 1: -0.220776
  Feature 2: 0.009188
  Feature 3: -0.006439


In [12]:
def predict_with_exposure(model, X, exposure):
    """
    Prédit en prenant en compte l'exposition
    
    Args:
        model: Le modèle entraîné
        X: Features pour la prédiction
        exposure: Valeurs d'exposition
    
    Returns:
        Prédictions ajustées selon l'exposition (probabilités)
    """
    # Obtenir directement les probabilités ajustées par l'exposition
    y_pred_proba = model.predict_proba(X, exposure)
    
    return y_pred_proba

In [13]:
y_pred_proba = predict_with_exposure(model, X_test, exposure_test)
y_pred_proba

array([0.20598805, 0.17227476, 0.21249701, ..., 0.14897138, 0.18496866,
       0.20810552], shape=(32643,))

In [14]:

def evaluate_model(model, X_test, y_test, exposure_test):
    """
    Évalue le modèle en tenant compte de l'exposition
    """
    # Prédictions ajustées selon l'exposition (probabilités)
    y_pred_exposed = predict_with_exposure(model, X_test, exposure_test)
    
    # Convertir en valeurs binaires pour l'évaluation (seuil à 0.5)
    y_pred_binary = (y_pred_exposed >= 0.5).astype(int)
    
    # Calculer les métriques
    accuracy = accuracy_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary, zero_division=0)
    recall = recall_score(y_test, y_pred_binary, zero_division=0)
    f1 = f1_score(y_test, y_pred_binary, zero_division=0)
    
    print("Performance du modèle:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1 Score: {f1:.4f}")
    
    # Afficher quelques exemples de prédictions
    print("\nExemples de prédictions (10 premières observations):")
    for i in range(min(10, len(y_test))):
        print(f"  Obs {i+1}: Réel={y_test[i]}, Proba={y_pred_exposed[i]:.4f}, Prédit={y_pred_binary[i]}, Exposition={exposure_test[i]:.2f}")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [15]:
evaluate_model(model, X_test, y_test, exposure_test)

Performance du modèle:
  Accuracy: 0.8879
  Precision: 0.0000
  Recall: 0.0000
  F1 Score: 0.0000

Exemples de prédictions (10 premières observations):
  Obs 1: Réel=0, Proba=0.2060, Prédit=0, Exposition=1.00
  Obs 2: Réel=1, Proba=0.1723, Prédit=0, Exposition=1.00
  Obs 3: Réel=1, Proba=0.2125, Prédit=0, Exposition=1.00
  Obs 4: Réel=0, Proba=0.0814, Prédit=0, Exposition=0.40
  Obs 5: Réel=1, Proba=0.1979, Prédit=0, Exposition=1.00
  Obs 6: Réel=0, Proba=0.2112, Prédit=0, Exposition=1.00
  Obs 7: Réel=0, Proba=0.1667, Prédit=0, Exposition=1.00
  Obs 8: Réel=0, Proba=0.2047, Prédit=0, Exposition=1.00
  Obs 9: Réel=1, Proba=0.2133, Prédit=0, Exposition=1.00
  Obs 10: Réel=0, Proba=0.2099, Prédit=0, Exposition=1.00


{'accuracy': 0.8878779523940814, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}

In [16]:
features = ['DriverAge', 'Gender', 'VehiculeUsage']
target="Sinistre"
X_train, X_test, y_train, y_test, exposure_train, exposure_test, scaler = prepare_data(df, features, target)
# # Entraîner le modèle avec l'exposition comme poids
model = train_model(X_train, y_train, exposure_train)
# 
# # Évaluer le modèle
metrics = evaluate_model(model, X_test, y_test, exposure_test)


Coefficients du modèle:
  Biais: -1.491195
  Feature 1: -0.116728
  Feature 2: 0.012637
  Feature 3: -0.000015
Performance du modèle:
  Accuracy: 0.8879
  Precision: 0.0000
  Recall: 0.0000
  F1 Score: 0.0000

Exemples de prédictions (10 premières observations):
  Obs 1: Réel=0, Proba=0.2060, Prédit=0, Exposition=1.00
  Obs 2: Réel=1, Proba=0.1723, Prédit=0, Exposition=1.00
  Obs 3: Réel=1, Proba=0.2125, Prédit=0, Exposition=1.00
  Obs 4: Réel=0, Proba=0.0814, Prédit=0, Exposition=0.40
  Obs 5: Réel=1, Proba=0.1979, Prédit=0, Exposition=1.00
  Obs 6: Réel=0, Proba=0.2112, Prédit=0, Exposition=1.00
  Obs 7: Réel=0, Proba=0.1667, Prédit=0, Exposition=1.00
  Obs 8: Réel=0, Proba=0.2047, Prédit=0, Exposition=1.00
  Obs 9: Réel=1, Proba=0.2133, Prédit=0, Exposition=1.00
  Obs 10: Réel=0, Proba=0.2099, Prédit=0, Exposition=1.00
